# Техническое задание Spinberry

**Описание игры:** слот-машина на трех барабанах с одной платежной линией (классический однорукий бандит). Выплата идет за 3 символа на линии

Символы: Вишня, Персик, Черешня, Манго, Апельсин, Банан
Выплата за 3 символа (в ставках):
Вишня: x0.5
Персик: x0.5
Черешня: x1
Манго: x1
Апельсин: x2
Банан: x5

**Определение RTP:** RTP (= Return to player) - величина, которая определяется для слот-машины как “Суммарный выигрыш за все спины” / “Суммарную ставку за все спины” * 100%




**Задача:** создать набор лент для заданной слот-машины, чтобы RTP равнялось 95% (погрешность +- 0.5%). Все символы должны использоваться, и все комбинации должны хоть когда-нибудь выплачиваться (т.е. нельзя сделать ленты только на вишенках). Предоставить подтверждающую статистику

In [ ]:
import pandas as pd

symbols = ["Cherry", "Peach", "Grape", "Mango", "Orange", "Banana"]
payouts = {"Cherry": 0.5, "Peach": 0.5, "Grape": 1, "Mango": 1, "Orange": 2, "Banana": 5}

reel_size = 100
target_rtp = 0.95

In [ ]:
def calculate_rtp_optimized(reel_counts, payouts, reel_size):
      """
    Calculate the Return to Player (RTP) for the current reel distribution.

    Args:
        reel_counts (dict): A dictionary with symbols as keys and their counts as values.
        payouts (dict): A dictionary with symbols as keys and their payout multipliers as values.
        reel_size (int): Total number of slots on the reel.

    Returns:
        float: RTP value as a percentage.
    """
    rtp = 0
    for symbol, count in reel_counts.items():
        # Вероятность того, что символ выпадет на всех трех барабанах
        prob = (count / reel_size) ** 3
        # Увеличиваем RTP с учетом вероятности и выплаты для символа
        rtp += prob * payouts[symbol]
    return rtp * 100  # Конвертация в проценты

In [ ]:
def probability_weighted_adjustment(target_rtp, payouts, reel_size):
      """
    Create an initial distribution of symbols based on payouts and probabilities.

    Args:
        target_rtp (float): Target RTP value as a decimal.
        payouts (dict): A dictionary with symbols as keys and their payout multipliers as values.
        reel_size (int): Total number of slots on the reel.

    Returns:
        dict: Initial distribution of symbols.
    """
    total_payout = sum(payouts.values())
    reel_counts = {
        symbol: max(1, int(reel_size * (payouts[symbol] / total_payout))) for symbol in payouts
    }

    # Корректировка с учетом вероятностей
    for symbol in payouts:
        weight = payouts[symbol] / total_payout
        reel_counts[symbol] = max(1, int(reel_counts[symbol] * (1 - weight)))

    # Обеспечение общей суммы символов, равной размеру барабана
    total_adjusted = sum(reel_counts.values())
    difference = reel_size - total_adjusted
    for symbol in sorted(payouts, key=payouts.get, reverse=(difference > 0)):
        reel_counts[symbol] += difference
        if sum(reel_counts.values()) == reel_size:
            break

    return reel_counts

In [ ]:
def refine_rtp_to_target(reel_counts, payouts, target_rtp, reel_size, tolerance=0.005):
      """
    Refine the reel distribution to achieve the target RTP.

    Args:
        reel_counts (dict): Initial reel distribution.
        payouts (dict): A dictionary with symbols as keys and their payout multipliers as values.
        target_rtp (float): Target RTP value as a decimal.
        reel_size (int): Total number of slots on the reel.
        tolerance (float): Allowed deviation from the target RTP.

    Returns:
        tuple: Refined reel distribution and achieved RTP.
    """
    current_rtp = calculate_rtp_optimized(reel_counts, payouts, reel_size)

    # Итеративно корректируем, пока RTP не окажется в пределах целевого значения
    while abs(current_rtp - target_rtp * 100) > tolerance:
        if current_rtp > target_rtp * 100:
            # Уменьшаем символы с высокими выплатами
            for symbol in sorted(payouts, key=payouts.get, reverse=True):
                if reel_counts[symbol] > 1:
                    reel_counts[symbol] -= 1
                    break
        else:
            # Увеличиваем символы с низкими выплатами
            for symbol in sorted(payouts, key=payouts.get):
                reel_counts[symbol] += 1
                break

        # Перерасчёт RTP после каждой корректировки
        current_rtp = calculate_rtp_optimized(reel_counts, payouts, reel_size)

    return reel_counts, current_rtp

In [ ]:
# Настройка барабанов с использованием предыдущего подхода
initial_reel_counts = probability_weighted_adjustment(target_rtp, payouts, reel_size)

# Уточнение до целевого RTP
final_reel_counts, final_rtp = refine_rtp_to_target(initial_reel_counts, payouts, target_rtp, reel_size)

# Создание DataFrame для отображения распределения
final_reel_distribution = pd.DataFrame.from_dict(final_reel_counts, orient="index", columns=["Count"])
final_reel_distribution["Probability"] = final_reel_distribution["Count"] / reel_size
final_reel_distribution["Payout"] = final_reel_distribution.index.map(payouts)
final_reel_distribution["Expected Contribution"] = (
    final_reel_distribution["Probability"] ** 3 * final_reel_distribution["Payout"]
)

# Вывод результатов
print("Final Reel Distribution:")
print(final_reel_distribution)
print(f"\nFinal RTP: {final_rtp}%")


Final Reel Distribution:
        Count  Probability  Payout  Expected Contribution
Cherry     78         0.78     0.5               0.237276
Peach       4         0.04     0.5               0.000032
Grape       9         0.09     1.0               0.000729
Mango       9         0.09     1.0               0.000729
Orange     16         0.16     2.0               0.008192
Banana     52         0.52     5.0               0.703040

Final RTP: 94.99980000000001%


ручная проверка

In [ ]:
manual_rtp = 0
for symbol, count in final_reel_counts.items():
    prob = (count / reel_size) ** 3
    manual_rtp += prob * payouts[symbol]
manual_rtp *= 100

print(f"Manual RTP: {manual_rtp}%")


Manual RTP: 94.99980000000001%
